### Language Translation 

Since some other descriptions or titles are in deutsch or french, they are translated to english using the following spark pipleine. The cleaned dataframe is stored as parquet files for the modelling phase

In [2]:
import string
import pandas as pd
import numpy as np
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

In [3]:
# Start Spark Session with Spark NLP
spark = sparknlp.start()

In [4]:
#read the dataframe
cleaned_df = pd.read_csv('../Data/cleanedpandasDf.csv')
cleaned_test_df = pd.read_csv('../Data/cleanedpandastestDf.csv')

In [ ]:
#converts the raw texts into a document format 
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

## Sentence Detection using Deep Learning
sentencerDL = SentenceDetectorDLModel()\
.pretrained("sentence_detector_dl", "xx")\
.setInputCols(["document"])\
 
#the language translation model
marian = MarianTransformer.pretrained("opus_mt_mul_en", "xx").setInputCols(["sentences"]).setOutputCol("translation")

#building the pipeline for translation
marian_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])

In [ ]:
#convert the pandas df as a spark dataframe.
sparkDf = spark.createDataFrame(cleaned_df)
sparkTestDf = spark.createDataFrame(cleaned_test_df)

In [ ]:
#translate the dataframe and the test dataframe.
result = marian_pipeline.fit(sparkDf).transform(sparkDf)
result_test = marian_pipeline.fit(sparkTestDf).transform(sparkTestDf)

In [ ]:
# the spark dataframes are converted to be pandas type frame and saved as pickle files (just if needed in this format)
pandasdf = result.select("level", "title", "text", "description", "translation.result").toPandas()
pandasdf.to_csv("../Data/translatedCleanedDf.csv") 

In [ ]:
pandasdf_test = result.select("title", "text", "description", "translation.result").toPandas()
pandasdf_test.to_csv("../Data/translatedCleanedTestDf.pkl")

In [ ]:
#spark.stop()